write bitstream

In [ ]:
from pynq import Overlay
from pynq import Xlnk
import pprint
import matplotlib.pyplot as plt
import numpy as np
import cv2
from time import time
import math
xlnk = Xlnk()
overlay = Overlay('./ORB_zcu104.bit')

test

In [ ]:
dma_cfg = overlay.axi_dma_cfg
dma_data = overlay.axi_dma_data

dma_cfg.sendchannel.start()
dma_data.sendchannel.start()
dma_data.recvchannel.start()

In [ ]:
img_ori = cv2.imread('000000.png')
height = img_ori.shape[0]
width = img_ori.shape[1]
img_gray = cv2.cvtColor(img_ori,cv2.COLOR_BGR2GRAY)

src_buf = xlnk.cma_array(shape=(width*height,), dtype=np.uint8)
view=np.frombuffer(src_buf,dtype = np.uint8,count = -1)
np.copyto(view,img_gray.ravel(),casting='same_kind')
des_buf = xlnk.cma_array(shape=(8192,16), dtype=np.uint32)
cfg_src_buf = xlnk.cma_array(shape=(4,), dtype=np.uint32)
scale = pow(1.2, 1)
cfg_src_buf[0] = width
cfg_src_buf[1] = height
cfg_src_buf[2] = scale * pow(2, 14)
cfg_src_buf[3] = 1 / scale * pow(2, 14)

In [ ]:
dma_cfg.sendchannel.transfer(cfg_src_buf)
dma_data.sendchannel.transfer(src_buf)
dma_data.recvchannel.transfer(des_buf)

dma_cfg.sendchannel.wait()
dma_data.sendchannel.wait()
dma_data.recvchannel.wait()

des_buf.flush()
bytes_read=dma_data.mmio.read(0x58)
featurePointsNum = int(bytes_read/64) - 1
print(str(featurePointsNum)+'  pixels are detected')

In [ ]:
for i in range(2, featurePointsNum):
    row_ind = des_buf[i][0] >> 16 & 0b111111111
    col_ind = (des_buf[i][0] >> 25) + ((des_buf[i][1] & 0b1111) << 7)
    angle =( des_buf[i][0] >> 7 ) & 0b111111111
    score = des_buf[i][0] & 0b1111111
    cv2.circle(img_ori,(np.int(col_ind*scale),np.int(row_ind*scale)),1,(0,0,255),-1)
#     for j in range(8):
#         print((des_buf[i][j+1]>>4)+((des_buf[i][j+2]& 0b1111)<<28))

    print(row_ind*scale, col_ind*scale)
    print(angle/32)
    print(score)

In [ ]:
plt.figure(figsize=(10,7.5))
plt.imshow(img_ori, cmap='gray')